### Kuhn Poker Testing (Prompt-Based)

In [ ]:
# %pip install open_spiel
%pip install datasets
%pip install transformers
%pip install --upgrade transformers
%pip install -U datasets fsspec

In [ ]:
#%pip install sae-lens

In [2]:
import torch
print(torch.cuda.is_available())

False


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

#torch.set_default_device("cuda")

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B", device_map="cpu")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B", device_map="cpu")

c:\GitHub\MechInterpGameTheory\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  6.00s/it]
c:\GitHub\MechInterpGameTheory\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanis\.cache\huggingface\hub\models--Qwen--Qwen3-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer

In [5]:
from datasets import load_dataset
import json
# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("the-acorn-ai/kuhn-poker-Qwen3-32B-5000-sft")

c:\GitHub\MechInterpGameTheory\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanis\.cache\huggingface\hub\datasets--the-acorn-ai--kuhn-poker-Qwen3-32B-5000-sft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 25548/25548 [00:00<00:00, 104050.79 examples/s

In [7]:
import re
game_state = ds["train"][0]["messages"][0]["content"] + " Make sure to keep your response concise and under 100 words"
print(game_state)

inputs = tokenizer(game_state, return_tensors="pt")
inputs = inputs.to("cpu") # or "cuda" if you have a GPU (it is much faster with CUDA)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=False,
    temperature=0.7,
    top_k=30,
    top_p=0.9,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)

output_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("\n")
print(f"Raw output answer: {output_answer}")
print(re.findall(r'\\boxed{(.*?)}', output_answer)[1])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are playing a two-player zero-sum game. Make valid actions to win. Observation: 
[GAME] You are Player 1 in a 5 round game of Kuhn Poker.
Game Rules:
- Kuhn Poker uses a 3-card deck with J, Q, K (J lowest, K highest)
- Each player antes 1 chip and receives 1 card each round
- Game continues for 5 rounds
- The player with the most chips after all rounds wins

Action Rules:
- '[check]': Pass without betting (only if no bet is on the table)
- '[bet]': Add 1 chip to the pot (only if no bet is on the table)
- '[call]': Match an opponent's bet by adding 1 chip to the pot
- '[fold]': Surrender your hand and let your opponent win the pot


[GAME] Starting round 1 out of 5 rounds.
Your card is: Q
Your available actions are: [check], [bet].
Please reason step by step, and put your final answer within \boxed{}. Make sure to keep your response concise and under 100 words


Raw output answer: You are playing a two-player zero-sum game. Make valid actions to win. Observation: 
[GAME] You are Pla